<a href="https://colab.research.google.com/github/rsotop/BPMN_GPT/blob/main/Redesign_BPMN_GPT_Shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Business process redesign using BPMN and ChatGPT

This experiment consists of generating a proposal for the redesign of a process (to-be) using ChatGPT, from an XML file (.bpmn) that represents the current situation of a process (as-is) generated by the tool. [BPMN.iO]("https://demo.bpmn.io//new//") de Camunda.


### GPT configuration
#### Load the API key and the relevant Python libraries.

In [2]:
!pip install openai

import openai
import os
openai.api_key = 'sk-...'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.4 MB/s eta 0:00:00


#### helper function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs:

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # este es el grado de aleatoriedad de la salida del modelo
    )
    return response.choices[0].message["content"]

#### Táctica: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

### BPMN.io environment configuration

This function allows you to view the resulting BPMN diagram in HTML format

In [35]:
from IPython.display import HTML

def generate_bpmn_html(bpmn_xml):
    # Código HTML para cargar bpmn-js y mostrar el diagrama
    html_content = f"""
    <html>
        <head>
            <title>BPMN Viewer</title>
            <script src="https://cdn.jsdelivr.net/npm/bpmn-js/dist/bpmn-viewer.production.min.js"></script>
            <style>
                #canvas {{
                    height: 600px;
                    width: 100%;
                }}
            </style>
        </head>
        <body>
            <div id="canvas"></div>
            <script>
                const bpmnXML = `{bpmn_xml}`;
                const viewer = new BpmnJS({{ container: '#canvas' }});

                viewer.importXML(bpmnXML, function(err) {{
                    if (err) {{
                        console.error('Error rendering BPMN diagram', err);
                    }} else {{
                        console.log('BPMN diagram rendered.');
                    }}
                }});
            </script>
        </body>
    </html>
    """

    return html_content

#### XML object (.bpmn) generated in [BPMN.iO]("https://demo.bpmn.io//new//").

In [37]:
bpmnXmlAsIs = f"""
<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" xmlns:di="http://www.omg.org/spec/DD/20100524/DI" id="Definitions_0e75zcs" targetNamespace="http://bpmn.io/schema/bpmn" exporter="bpmn-js (https://demo.bpmn.io)" exporterVersion="12.0.0">
  <bpmn:process id="Process_0hg2idu" isExecutable="false">
    <bpmn:startEvent id="StartEvent_05yz5xt">
      <bpmn:outgoing>Flow_0t51g4g</bpmn:outgoing>
    </bpmn:startEvent>
    <bpmn:task id="Activity_1dcam6f" name="Register">
      <bpmn:incoming>Flow_0t51g4g</bpmn:incoming>
      <bpmn:outgoing>Flow_1cq0yap</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_0t51g4g" sourceRef="StartEvent_05yz5xt" targetRef="Activity_1dcam6f" />
    <bpmn:task id="Activity_083ly9r" name="Analyze defect">
      <bpmn:incoming>Flow_1cq0yap</bpmn:incoming>
      <bpmn:outgoing>Flow_1uwzowl</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1cq0yap" sourceRef="Activity_1dcam6f" targetRef="Activity_083ly9r" />
    <bpmn:task id="Activity_1qxfuyi" name="Repair (complex)">
      <bpmn:incoming>Flow_1uwzowl</bpmn:incoming>
      <bpmn:outgoing>Flow_0gl6qbs</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1uwzowl" sourceRef="Activity_083ly9r" targetRef="Activity_1qxfuyi" />
    <bpmn:exclusiveGateway id="Gateway_1v7g0s4">
      <bpmn:incoming>Flow_0gl6qbs</bpmn:incoming>
      <bpmn:outgoing>Flow_02cc8b2</bpmn:outgoing>
      <bpmn:outgoing>Flow_1990s0p</bpmn:outgoing>
      <bpmn:outgoing>Flow_08m238a</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:sequenceFlow id="Flow_0gl6qbs" sourceRef="Activity_1qxfuyi" targetRef="Gateway_1v7g0s4" />
    <bpmn:task id="Activity_192grsq" name="Inform user">
      <bpmn:incoming>Flow_02cc8b2</bpmn:incoming>
      <bpmn:outgoing>Flow_08lguql</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_02cc8b2" sourceRef="Gateway_1v7g0s4" targetRef="Activity_192grsq" />
    <bpmn:task id="Activity_01h1d8v" name="Test repair">
      <bpmn:incoming>Flow_1990s0p</bpmn:incoming>
      <bpmn:outgoing>Flow_1s8b3hl</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1990s0p" sourceRef="Gateway_1v7g0s4" targetRef="Activity_01h1d8v" />
    <bpmn:task id="Activity_1wi5hg7" name="Repair (simple)">
      <bpmn:incoming>Flow_08m238a</bpmn:incoming>
      <bpmn:outgoing>Flow_1q0od66</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_08m238a" sourceRef="Gateway_1v7g0s4" targetRef="Activity_1wi5hg7" />
    <bpmn:task id="Activity_017ggai" name="Restart repair">
      <bpmn:incoming>Flow_1q0od66</bpmn:incoming>
      <bpmn:outgoing>Flow_1r7dx1w</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1q0od66" sourceRef="Activity_1wi5hg7" targetRef="Activity_017ggai" />
    <bpmn:exclusiveGateway id="Gateway_0dsfhde">
      <bpmn:incoming>Flow_1r7dx1w</bpmn:incoming>
      <bpmn:incoming>Flow_08lguql</bpmn:incoming>
      <bpmn:incoming>Flow_1s8b3hl</bpmn:incoming>
      <bpmn:outgoing>Flow_16s0nqt</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:sequenceFlow id="Flow_1r7dx1w" sourceRef="Activity_017ggai" targetRef="Gateway_0dsfhde" />
    <bpmn:sequenceFlow id="Flow_08lguql" sourceRef="Activity_192grsq" targetRef="Gateway_0dsfhde" />
    <bpmn:sequenceFlow id="Flow_1s8b3hl" sourceRef="Activity_01h1d8v" targetRef="Gateway_0dsfhde" />
    <bpmn:task id="Activity_096egnc" name="Archive repair">
      <bpmn:incoming>Flow_16s0nqt</bpmn:incoming>
      <bpmn:outgoing>Flow_05uz9lz</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_16s0nqt" sourceRef="Gateway_0dsfhde" targetRef="Activity_096egnc" />
    <bpmn:endEvent id="Event_0tyccj5">
      <bpmn:incoming>Flow_05uz9lz</bpmn:incoming>
    </bpmn:endEvent>
    <bpmn:sequenceFlow id="Flow_05uz9lz" sourceRef="Activity_096egnc" targetRef="Event_0tyccj5" />
  </bpmn:process>
  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_0hg2idu">
      <bpmndi:BPMNShape id="_BPMNShape_StartEvent_2" bpmnElement="StartEvent_05yz5xt">
        <dc:Bounds x="192" y="292" width="36" height="36" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_1dcam6f_di" bpmnElement="Activity_1dcam6f">
        <dc:Bounds x="310" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_083ly9r_di" bpmnElement="Activity_083ly9r">
        <dc:Bounds x="480" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_1qxfuyi_di" bpmnElement="Activity_1qxfuyi">
        <dc:Bounds x="650" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Gateway_1v7g0s4_di" bpmnElement="Gateway_1v7g0s4" isMarkerVisible="true">
        <dc:Bounds x="825" y="285" width="50" height="50" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_192grsq_di" bpmnElement="Activity_192grsq">
        <dc:Bounds x="1000" y="100" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_01h1d8v_di" bpmnElement="Activity_01h1d8v">
        <dc:Bounds x="1000" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_1wi5hg7_di" bpmnElement="Activity_1wi5hg7">
        <dc:Bounds x="920" y="430" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_017ggai_di" bpmnElement="Activity_017ggai">
        <dc:Bounds x="1090" y="430" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Gateway_0dsfhde_di" bpmnElement="Gateway_0dsfhde" isMarkerVisible="true">
        <dc:Bounds x="1235" y="285" width="50" height="50" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_096egnc_di" bpmnElement="Activity_096egnc">
        <dc:Bounds x="1350" y="270" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Event_0tyccj5_di" bpmnElement="Event_0tyccj5">
        <dc:Bounds x="1522" y="292" width="36" height="36" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNEdge id="Flow_0t51g4g_di" bpmnElement="Flow_0t51g4g">
        <di:waypoint x="228" y="310" />
        <di:waypoint x="310" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1cq0yap_di" bpmnElement="Flow_1cq0yap">
        <di:waypoint x="410" y="310" />
        <di:waypoint x="480" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1uwzowl_di" bpmnElement="Flow_1uwzowl">
        <di:waypoint x="580" y="310" />
        <di:waypoint x="650" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_0gl6qbs_di" bpmnElement="Flow_0gl6qbs">
        <di:waypoint x="750" y="310" />
        <di:waypoint x="825" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_02cc8b2_di" bpmnElement="Flow_02cc8b2">
        <di:waypoint x="850" y="285" />
        <di:waypoint x="850" y="140" />
        <di:waypoint x="1000" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1990s0p_di" bpmnElement="Flow_1990s0p">
        <di:waypoint x="875" y="310" />
        <di:waypoint x="1000" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_08m238a_di" bpmnElement="Flow_08m238a">
        <di:waypoint x="850" y="335" />
        <di:waypoint x="850" y="470" />
        <di:waypoint x="920" y="470" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1q0od66_di" bpmnElement="Flow_1q0od66">
        <di:waypoint x="1020" y="470" />
        <di:waypoint x="1090" y="470" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1r7dx1w_di" bpmnElement="Flow_1r7dx1w">
        <di:waypoint x="1190" y="470" />
        <di:waypoint x="1260" y="470" />
        <di:waypoint x="1260" y="335" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_08lguql_di" bpmnElement="Flow_08lguql">
        <di:waypoint x="1100" y="140" />
        <di:waypoint x="1260" y="140" />
        <di:waypoint x="1260" y="285" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1s8b3hl_di" bpmnElement="Flow_1s8b3hl">
        <di:waypoint x="1100" y="310" />
        <di:waypoint x="1235" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_16s0nqt_di" bpmnElement="Flow_16s0nqt">
        <di:waypoint x="1285" y="310" />
        <di:waypoint x="1350" y="310" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_05uz9lz_di" bpmnElement="Flow_05uz9lz">
        <di:waypoint x="1450" y="310" />
        <di:waypoint x="1522" y="310" />
      </bpmndi:BPMNEdge>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>
</bpmn:definitions>
"""

In [38]:
html = generate_bpmn_html(bpmnXmlAsIs)
HTML(html)

In [39]:
prompt = f"""
From the XML object (.bpmn) provided, which represents the current \
situation of a business process (as-is), it generates a redesign \
proposal (to-be). You can restructure activities, introduce or remove \
activities, propose other resources such as automation. You can \
compare this process to other processes to give hints on how to make \
it better. Be creative, however, build your judgement based on facts as \
much as possible

\"\"\"{bpmnXmlAsIs}\"\"\"
"""
response = get_completion(prompt)
print(response)

Redesign Proposal:

The current business process has several tasks that can be optimized to improve efficiency and reduce the time taken to complete the process. Here are some suggestions for redesigning the process:

1. Introduce automation: The process can be automated to reduce the time taken to complete tasks. For example, the task "Analyze defect" can be automated using machine learning algorithms to identify the defect and suggest the appropriate repair method.

2. Combine tasks: The tasks "Test repair" and "Restart repair" can be combined into a single task to reduce the number of steps in the process.

3. Remove unnecessary tasks: The task "Inform user" can be removed as it does not add any value to the process.

4. Introduce parallel processing: The tasks "Repair (complex)" and "Repair (simple)" can be performed in parallel to reduce the time taken to complete the process.

5. Improve communication: The process can be improved by introducing better communication channels betwe

In [40]:
prompt2 = f"""
From the redesign proposal (to-be), generate a new XML object \
(.bpmn) that can be interpreted by the Camunda Modeler tool.

\"\"\"{response}\"\"\"
"""
bpmnXmlToBe = get_completion(prompt2)
print(bpmnXmlToBe) 

<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL"
                  xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
                  xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
                  xmlns:di="http://www.omg.org/spec/DD/20100524/DI"
                  id="Definitions_1"
                  targetNamespace="http://bpmn.io/schema/bpmn"
                  exporter="Camunda Modeler"
                  exporterVersion="4.5.0">
  <bpmn:process id="Process_1" name="Redesigned Process">
    <bpmn:startEvent id="StartEvent_1">
      <bpmn:outgoing>SequenceFlow_1</bpmn:outgoing>
    </bpmn:startEvent>
    <bpmn:task id="Task_1" name="Analyze defect">
      <bpmn:incoming>SequenceFlow_1</bpmn:incoming>
      <bpmn:outgoing>SequenceFlow_2</bpmn:outgoing>
    </bpmn:task>
    <bpmn:task id="Task_2" name="Repair (complex)">
      <bpmn:incoming>SequenceFlow_2</bpmn:incoming>
      <bpmn:outgoing>SequenceFlow_3</bpm

In [41]:
html = generate_bpmn_html(bpmnXmlToBe)
HTML(html)

#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.

### References:
- [GPT-4: A Paradigm Shift in Business Process Improvement](https://www.linkedin.com/pulse/gpt-4-paradigm-shift-business-process-improvement-ihar-suvorau?trackingId=tbLExbPqSHuBUhElhfMivA%3D%3D/?trackingId=tbLExbPqSHuBUhElhfMivA==)
- [ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines)
- [Camunda Academy](https://academy.camunda.com/)
- [BPMN.iO](https://bpmn.io/)
